In [1]:
## code authors: katharina lichter, philip kollmannsberger, university and university hospital of wuerzburg, 2022.

## this code is used in "Ultrastructural analysis of wildtype and RIM1α knock-out active zones in a large cortical synapse"
## by k lichter, mm paul, m pauli, s schoch, p kollmannsberger, c stigloher, m heckmann, a-l sirén

## notebook for calculation of nearest neighbour (NN) distances in the synaptic vesicle (SV) pool

# please import relevant python packages via anaconda navigator or anaconda prompt.
import numpy as np
import math as math
import scipy as sp
from shapely.geometry import LineString, MultiPoint, Point
from scipy.spatial import distance
from sklearn import neighbors
from sklearn.neighbors import NearestNeighbors

##############################################################################

## please import active zone (AZ) information which is extracted from individual IMOD models (https://bio3d.colorado.edu/imod/). 
# please note that coordinate data are provided in text files. in case of SV data, the SV radius represents a separate column.

# step 1: load the coordinates.

pts = np.loadtxt(fname = 'az-test_sv-pool.txt', usecols = [1,2,3,4,5,6,7]);
linepts = np.loadtxt(fname = 'az-test_area.txt', usecols = [1,2,3,4,5,6]);

# step 2: add the pixel size of the tomogram. this information can be found in your original tilt series.
px_factor = 0.287

## step 3: calculation of NN euclidean distances within the SV pool. 
# calculation of euclidean distances from each SV to all other SVs

r = np.array(pts[:,6])
repetitions = r.size

rarray = np.reshape(r, (repetitions, 1))

rmatrix1 = np.tile(rarray, repetitions)
rmatrix2 = np.tile(r,(repetitions, 1))

scale = np.tile(np.array(px_factor), (repetitions, repetitions))

vesicle = MultiPoint(pts[:,3:6]);

D = distance.squareform(distance.pdist(vesicle));

rmatrix = np.add(rmatrix1, rmatrix2)
nearest = np.multiply(np.subtract(D, rmatrix), scale)

# individual distance matrices for each SV             
dnearest = np.where(nearest<0, 0, nearest);
(dnearest)

# calculation of the 5 nearest neighbouring SVs (index, absolute distance)
nn = NearestNeighbors(n_neighbors=6, metric='precomputed')
nn.fit(dnearest)

distances, indexes = nn.kneighbors(dnearest)

np.set_printoptions(threshold=np.inf)

knearest = nn.kneighbors(dnearest)

with open("az-test_kn5_index.txt", 'w') as f:
    print(knearest, file=f)
    
print('')

with open("az-test_kn5_dist.txt", 'w') as f:
    print(distances, file=f)